In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

In [2]:
symbols = ["H", "H", "H"]
coordinates = np.array([[0.0102, 0.0442, 0.0],[0.9867, 1.6303, 0.0],[1.8720, -0.0085, 0.0]])

In [3]:
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge = 1) #carga positiva +1, ou seja um eletron 
#foi removido

In [4]:
print(qubits) #Quantos qubits serao necessarios

6


In [5]:
hf = qchem.hf_state(electrons = 2, orbitals = 6) #Hartree-forck state =hf, orbitals = numero de qubits

In [6]:
print(hf) #Estado na representacao de Jordan-Wigner

[1 1 0 0 0 0]


In [7]:
num_wires = qubits
dev = qml.device("default.qubit", wires = num_wires)

In [8]:
@qml.qnode(dev)
def exp_energy(state):
    
    qml.BasisState(np.array(state), wires = range(num_wires))
    return qml.expval(hamiltonian)

In [9]:
exp_energy(hf)

tensor(-1.24655016, requires_grad=True)

In [10]:
def ansatz(params):
    qml.BasisState(hf, wires = range(num_wires))
    qml.DoubleExcitation(params[0], wires = [0, 1, 2, 3])
    qml.DoubleExcitation(params[1], wires = [0, 1, 4, 5])

In [11]:
@qml.qnode(dev)
def cost_function(params):
    ansatz(params)
    return qml.expval(hamiltonian)

In [12]:
cost_function([0.1, 0.1])

tensor(-1.26796721, requires_grad=True)

In [13]:
opt = qml.GradientDescentOptimizer(stepsize = 0.4)
theta = np.array([0.0, 0.0], requires_grad = True)

energy = [cost_function(theta)]
angle = [theta]
max_interations = 20

for n in range(max_interations):
    theta, prev_energy = opt.step_and_cost(cost_function, theta)
    
    energy.append(cost_function(theta))
    angle.append(theta)
    
    if n%2 ==0:
        print(f"Step = {n}, energy = {energy[-1]:.8f} Ha" )

Step = 0, energy = -1.26070025 Ha
Step = 2, energy = -1.27115671 Ha
Step = 4, energy = -1.27365804 Ha
Step = 6, energy = -1.27425241 Ha
Step = 8, energy = -1.27439362 Ha
Step = 10, energy = -1.27442718 Ha
Step = 12, energy = -1.27443517 Ha
Step = 14, energy = -1.27443707 Ha
Step = 16, energy = -1.27443752 Ha
Step = 18, energy = -1.27443763 Ha


In [14]:
print("\n" f"Final ground energy: {energy[-1]:.8f} Ha")
print("\n" f"Final angle parameters: {theta[0]:.8f} {theta[1]:.8f}")


Final ground energy: -1.27443764 Ha

Final angle parameters: 0.19203468 0.19290335


In [15]:
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()

In [16]:
ground_state(theta)

tensor([ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09585862+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09586987+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.99076743+0.j,  0.        +0.j,  0.        +0.